In [1]:
import tensorflow as tf
import numpy as np
import math

Hint: Use dtype=tf.float64 if you want to have same precision as numpy for testing<br>
Hint: You migth wanna use tf.InterativeSession for convenience

1a: Create two random 0-d tensors x and y of any distribution. <br>
Create a TensorFlow object that returns x + y if x > y, and x - y otherwise. <br>
Hint: look up tf.cond() <br>
I do the first problem for you <br>

In [2]:
def task_1a_np(x, y):
    return np.where(x > y, x + y, x - y)

In [3]:
X = tf.placeholder(tf.float64, name="X")
Y = tf.placeholder(tf.float64, name="Y")
out = tf.cond(tf.greater(X, Y), lambda: tf.add(X, Y), lambda: tf.subtract(X, Y))

In [4]:
with tf.Session() as sess:
    for xx, yy in np.random.uniform(size=(50, 2)):
        actual = sess.run(out, feed_dict={X:xx, Y:yy})
        expected = task_1a_np(xx, yy)
        if actual != expected:
            print('Fail')
            # something something
    else:
        print('Success')

Success


1b: Create two 0-d tensors x and y randomly selected from the range [-1, 1).<br>
Return x + y if x < y, x - y if x > y, 0 otherwise.<br>
Hint: Look up tf.case().<br>

In [5]:
def task_1b_np(x, y):
    return np.select(condlist=[x < y, x > y],
                     choicelist=[x + y, x - y],
                     default=0)

In [6]:
X = tf.placeholder(tf.float64, name="X")
Y = tf.placeholder(tf.float64, name="Y")
out = tf.case({tf.less(X, Y): lambda: tf.add(X, Y), tf.greater(X, Y): lambda: tf.subtract(X, Y)},
              default=lambda: tf.constant(0, dtype=tf.float64))

In [7]:
with tf.Session() as sess:
    for xx, yy in np.random.uniform(low=-1, high=1, size=(150, 2)):
        actual = sess.run(out, feed_dict={X:xx, Y:yy})
        expected = task_1b_np(xx, yy)
        if actual != expected:
            print('Fail')
    else:
        print('Success')

Success


1c: Create the tensor x of the value [[0, -2, -1], [0, 1, 2]] <br>
and y as a tensor of zeros with the same shape as x. <br>
Return a boolean tensor that yields Trues if x equals y element-wise. <br>
Hint: Look up tf.equal(). <br>

In [8]:
def task_1c_np():
    x = np.array([[0, -2, -1], [0, 1, 2]])
    y = np.zeros_like(x)
    return x == y

In [9]:
X = tf.constant([[0, -2, -1], [0, 1, 2]], tf.float64, name="X")
Y = tf.zeros_like(X, name="Y")
out = tf.equal(X, Y)

In [10]:
with tf.Session() as sess:
    actual = sess.run(out)
    expected = task_1c_np()
    if np.array_equal(actual, expected):
        print('Success')
    else:
        print('Fail')

Success


1d:<br>
Get the indices of elements in x whose values are greater than 30.<br>
Hint: Use tf.where().<br>
Then extract elements whose values are greater than 30.<br>
Hint: Use tf.gather().<br>

In [11]:
def task_1d_np(x):
    return x[x > 30].reshape(-1, 1)

In [12]:
X = tf.placeholder(tf.float64, name="X")
indices = tf.where(X > 30)
out = tf.gather(X, indices)

In [13]:
with tf.Session() as sess:
    x = np.random.uniform(low=-100, high=100, size=150)
    actual = sess.run(out, feed_dict={X: x})
    expected = task_1d_np(x)
    if np.array_equal(actual, expected):
        print('Success')
    else:
        print('Fail')

Success


1e: Create a diagnoal 2-d tensor of size 6 x 6 with the diagonal values of 1,<br>
2, ..., 6<br>
Hint: Use tf.range() and tf.diag().<br>

In [14]:
def task_1e_np():
    return np.diag(np.arange(1, 7))

In [15]:
out = tf.diag(tf.range(start=1, limit=7))

In [16]:
with tf.Session() as sess:
    actual = sess.run(out)
    expected = task_1e_np()
    if np.array_equal(actual, expected):
        print('Success')
    else:
        print('Fail')

Success


1f: Create a random 2-d tensor of size 10 x 10 from any distribution.<br>
Calculate its determinant.<br>
Hint: Look at tf.matrix_determinant().<br>

In [17]:
def task_1f_np(x):
    return np.linalg.det(x)

In [18]:
X = tf.placeholder(tf.float64, name="X")
out = tf.matrix_determinant(X)

In [19]:
with tf.Session() as sess:
    x = np.random.uniform(low=-1, high=1, size=(10, 10))
    actual = sess.run(out, feed_dict={X: x})
    expected = task_1f_np(x)
    if np.isclose(actual, expected):
        print('Success')
    else:
        print('Fail')

Success


1g: Create tensor x with value [5, 2, 3, 5, 10, 6, 2, 3, 4, 2, 1, 1, 0, 9].<br>
Return the unique elements in x<br>
Hint: use tf.unique(). Keep in mind that tf.unique() returns a tuple.<br>

In [20]:
def task_1g_np():
    x = [5, 2, 3, 5, 10, 6, 2, 3, 4, 2, 1, 1, 0, 9]
    _, idx = np.unique(x, return_index=True)
    return np.take(x, sorted(idx))

In [21]:
X = [5, 2, 3, 5, 10, 6, 2, 3, 4, 2, 1, 1, 0, 9]
out, _ = tf.unique(X)

In [22]:
with tf.Session() as sess:
    actual = sess.run(out)
    expected = task_1g_np()
    if np.array_equal(actual, expected):
        print('Success')
    else:
        print('Fail')

Success


1h: Create two tensors x and y of shape 300 from any normal distribution,<br>
as long as they are from the same distribution.<br>
Use tf.cond() to return:<br>
- The mean squared error of (x - y) if the average of all elements in (x - y)<br>
  is negative, or<br>
- The sum of absolute value of all elements in the tensor (x - y) otherwise.<br>
Hint: see the Huber loss function in the lecture slides 3.<br>

In [23]:
def task_1h_np(x, y):
    average = np.mean(x - y)
    mse = np.mean((x - y) ** 2)
    asum = np.sum(np.abs(x - y))
    return mse if average < 0 else asum

In [24]:
X = tf.placeholder(tf.float64, name="X")
Y = tf.placeholder(tf.float64, name="Y")
average = tf.reduce_mean(X - Y)
mse = tf.reduce_mean((X - Y) ** 2)
asum = tf.reduce_sum(tf.abs(X - Y))
out = tf.cond(average < 0, lambda: mse, lambda: asum)

In [25]:
with tf.Session() as sess:
    x = np.random.randn(300)
    y = np.random.randn(300)
    actual = sess.run(out, feed_dict={X: x, Y: y})
    expected = task_1h_np(x, y)
    if np.all(np.isclose(actual, expected)):
        print('Success')
    else:
        print('Fail')

Success
